# Data Conversion

This notebook transforms our soundfiles into a numerical dataset for our models.

In [1]:
#Potentially needed installations
#!pip install librosa
# !pip install playsound
# !pip install tqdm

In [2]:
#Imports
import numpy as np
import pandas as pd
import librosa
#from playsound import playsound
#This is used to show a progress bar in the terminal. Helpful as the conversion can take a while.
from tqdm import tqdm
import concurrent.futures

## Preparation of the Overview Dataframe

In [3]:
def prepare_overview(list_of_files):
    """
    This function takes a list of files and creates a joint dataframe for audio file conversion
    """
    overview = None
    for file_path in list_of_files:
        with open(file_path, "r") as file:
            if overview is None:
                overview = pd.read_csv(file, sep="\t")
            else:
                overview = pd.concat([overview, pd.read_csv(file, sep="\t")], ignore_index=True)
    return overview

In [4]:
def fix_gender(gender):
    if gender == "male":
        return "male_masculine"
    elif gender == "female":
        return "female_feminine"
    else:
        return gender

In [13]:
def preprocess_overview(overview, folder_path):
    """
    This function takes a dataframe and preprocesses it for audio file conversion
    """
    # we only need files with a gender label
    overview = overview.dropna(subset=["gender"])
    # irrelevant columns for our analysis
    overview = overview.drop(columns=["variant", "segment", "sentence_id", "up_votes", "down_votes", "locale"])
    # fixing gender labels
    overview["gender"] = overview["gender"].apply(fix_gender)
    # dropping all files that are not simply male or female
    overview = overview[(overview["gender"] == "female_feminine") | (overview["gender"] == "male_masculine")]
    # randomly selecting one clip per client_id
    overview = overview.groupby("client_id").sample(n=1, random_state=27)
    # changing the path to reflect the location of the audio files
    overview["path"] = overview["path"].apply(lambda x: f"{folder_path}/{x}")
    return overview.reset_index(drop=True)

In [14]:
list_of_files = ["other_13.tsv", "validated_13.tsv", "other_14.tsv", "validated_14.tsv", "other_15.tsv", "validated_15.tsv", "other_16.tsv", "validated_16.tsv", "other_17.tsv", "validated_17.tsv","other_18.tsv", "other_19.tsv", "other_20.tsv", "other_21.tsv", "validated_18.tsv", "validated_19.tsv", "validated_20.tsv", "validated_21.tsv"]
overview = prepare_overview(list_of_files)
overview.info()
overview.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301362 entries, 0 to 301361
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   client_id        301362 non-null  object 
 1   path             301362 non-null  object 
 2   sentence         301362 non-null  object 
 3   up_votes         301362 non-null  int64  
 4   down_votes       301362 non-null  int64  
 5   age              225051 non-null  object 
 6   gender           202198 non-null  object 
 7   accents          213745 non-null  object 
 8   variant          0 non-null       float64
 9   locale           301362 non-null  object 
 10  segment          0 non-null       float64
 11  sentence_id      137662 non-null  object 
 12  sentence_domain  830 non-null     object 
dtypes: float64(2), int64(2), object(9)
memory usage: 29.9+ MB


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment,sentence_id,sentence_domain
count,301362,301362,301362,301362.000000,301362.000000,225051,202198,213745,0.0,301362,0.0,137662,830
unique,10410,301362,297565,NaN,NaN,9,8,811,NaN,1,NaN,136160,44
top,29b8505586cd43382cd695da6b943f401104be710a5b60...,common_voice_en_42221538.mp3,Lecture theatres may contain a number of black...,NaN,NaN,twenties,female,United States English,NaN,en,NaN,e3b8197117066a0e3776b1392de73c1f307e272ff5e0a8...,general
freq,19876,1,4,NaN,NaN,78441,63490,58130,NaN,301362,NaN,4,471
mean,NaN,NaN,NaN,0.219799,0.019332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,0.741439,0.147195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
overview.groupby("gender").size()

gender
do_not_wish_to_say      837
female                63490
female_feminine       34072
male                  63353
male_masculine        38130
non-binary              304
other                  1919
transgender              93
dtype: int64

In [16]:
# On my device, all audio files are in the same "clips" folder. Please change the path if your files are in a different location.
#overview = preprocess_overview(overview, "./clips")
overview=preprocess_overview(overview, "./clips")
overview.head()

,client_id,path,sentence,age,gender,accents,sentence_domain
0,00309ae51fe07824676d5a3b39837b0a8927c064b7a054...,./clips/common_voice_en_39637591.mp3,He is also called Giuseppe Cipriani by some au...,sixties,male_masculine,NaN,NaN
1,0061ede367759ac61d08c3d46fe4192b99311b720f00e4...,./clips/common_voice_en_37007558.mp3,Typically closely associated with various iron...,fourties,male_masculine,African-America,NaN
2,00722eccb4fa4a95b74a5920e97f8f13461774a5ee9d6c...,./clips/common_voice_en_41597051.mp3,"Dubbed a ""rave-up"", it was a feature of severa...",twenties,female_feminine,NaN,NaN
3,007e630babc69c73b5a0641e0034d1002c997b6bbf6521...,./clips/common_voice_en_38357954.mp3,After a month of heavy but inconclusive fighti...,teens,male_masculine,Canadian English,NaN
4,009cd4885d56fbbd6ecc8c0ab4dc9c027d5ad3dbb00c61...,./clips/common_voice_en_39578002.mp3,"With the end of Prohibition, Dwyer was finding...",fifties,male_masculine,Midatlantic,NaN


In [17]:
overview.describe(include="all")

,client_id,path,sentence,age,gender,accents,sentence_domain
count,1702,1702,1702,1680,1702,1007,5
unique,1702,1702,1702,9,2,202,3
top,ffe7de17aae0a78989589e0039a0ef0bba33e393ea3951...,./clips/common_voice_en_39635873.mp3,"English and French use the original spelling ""...",twenties,male_masculine,United States English,general
freq,1,1,1,756,1112,438,3


In [18]:
overview.groupby("gender").size()

gender
female_feminine     590
male_masculine     1112
dtype: int64

In [19]:
overview.groupby("age").size()

age
eighties       2
fifties      106
fourties     161
nineties       1
seventies     11
sixties       48
teens        242
thirties     353
twenties     756
dtype: int64

We have a slight class imbalance that we may want to remove/have to deal with within the model building.

## Conversion of the Audiofiles

In [25]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None, mono=True)

        # Feature extraction
        # MFCC (Mel-frequency cepstral coefficients) gives the timbre of the audio
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        
        # Chroma feature gives the harmonic content of the audio
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        
        # Spectral features
        # Spectral centroid gives the center of mass of the spectrum
        spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        # Spectral bandwidth gives the width of the spectrum
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        # Spectral contrast gives the difference in amplitude between peaks and valleys in the sound spectrum
        spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        # Spectral rolloff gives the frequency below which a certain percentage of the total spectral energy is contained
        spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        
        # Zero crossing rate gives the rate at which the signal changes sign
        zcr = librosa.feature.zero_crossing_rate(y)
        # Root mean square energy gives the energy of the signal (i.e., the loudness)
        rmse = librosa.feature.rms(y=y)

        # Combine features into a single feature vector
        features = np.hstack([
            np.mean(mfccs, axis=1), np.std(mfccs, axis=1),
            np.mean(chroma, axis=1), np.std(chroma, axis=1),
            np.mean(spec_centroid), np.std(spec_centroid),
            np.mean(spec_bw), np.std(spec_bw),
            np.mean(spec_contrast, axis=1), np.std(spec_contrast, axis=1),
            np.mean(spec_rolloff), np.std(spec_rolloff),
            np.mean(zcr), np.std(zcr),
            np.mean(rmse), np.std(rmse)
        ])

        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


In [26]:
def build_columns():
    columns = []
    columns += [f"mfcc_{i+1:02d}_mean" for i in range(20)]
    columns += [f"mfcc_{i+1:02d}_std" for i in range(20)]
    columns += [f"chroma_{i+1:02d}_mean" for i in range(12)]
    columns += [f"chroma_{i+1:02d}_std" for i in range(12)]
    columns += ["spec_centroid_mean", "spec_centroid_std"]
    columns += ["spec_bandwidth_mean", "spec_bandwidth_std"]
    columns += [f"spec_contrast_band_{i+1}_mean" for i in range(7)]
    columns += [f"spec_contrast_band_{i+1}_std" for i in range(7)]
    columns += ["spec_rolloff_mean", "spec_rolloff_std"]
    columns += ["zcr_mean", "zcr_std"]
    columns += ["rmse_mean", "rmse_std"]
    return columns

In [27]:
def extract_features_from_dataframe(df_paths, output_csv_path, parallel=True):
    features_list = []
    valid_indices = []

    paths = df_paths['path'].tolist()

    # Use ThreadPoolExecutor for parallel processing
    # This helps to speed up the feature extraction process
    if parallel:
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(extract_features, path): idx for idx, path in enumerate(paths)}
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(paths), desc="Extracting features"):
                result = future.result()
                if result is not None:
                    features_list.append(result)
                    valid_indices.append(futures[future])
    else:
        for idx, path in tqdm(enumerate(paths), total=len(paths), desc="Extracting features"):
            result = extract_features(path)
            if result is not None:
                features_list.append(result)
                valid_indices.append(idx)

    # Build features DataFrame
    columns = build_columns()
    features_df = pd.DataFrame(features_list, columns=columns)
    
    # Match features to original DataFrame
    merged_df = df_paths.iloc[valid_indices].reset_index(drop=True)
    final_df = pd.concat([merged_df, features_df], axis=1)

    # Save to CSV
    final_df.to_csv(output_csv_path, index=False)
    print(f"Saved extracted features to {output_csv_path}")

    return final_df

In [ ]:
#df=extract_features_from_dataframe(overview, "./data/data_1.csv", parallel=True)
df=extract_features_from_dataframe(overview_2, "./data/data_2.csv", parallel=True)
df.describe()

Extracting features:  29%|██▉       | 44872/153687 [47:22<4:08:15,  7.31it/s]  c:\Users\danie\anaconda3\envs\tensorflow\Lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Extracting features: 100%|██████████| 153687/153687 [2:29:43<00:00, 17.11it/s]  


Saved extracted features to ./data/data_2.csv


,mfcc_01_mean,mfcc_02_mean,mfcc_03_mean,mfcc_04_mean,mfcc_05_mean,mfcc_06_mean,mfcc_07_mean,mfcc_08_mean,mfcc_09_mean,mfcc_10_mean,...,spec_contrast_band_4_std,spec_contrast_band_5_std,spec_contrast_band_6_std,spec_contrast_band_7_std,spec_rolloff_mean,spec_rolloff_std,zcr_mean,zcr_std,rmse_mean,rmse_std
count,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,...,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000,153687.000000
mean,-423.785510,102.472514,-6.831261,25.223064,2.653685,2.205840,-8.246838,-7.198746,-7.167236,-7.834317,...,5.087988,5.142466,8.380170,7.935775,3852.102317,2301.920853,0.099244,0.086102,0.045506,0.048407
std,72.663292,23.937435,18.979706,14.314564,13.207778,12.381509,8.865790,11.223413,9.070593,7.244752,...,0.885814,1.199569,2.313839,3.358012,1264.732632,620.724959,0.033269,0.025126,0.026200,0.026480
min,-1131.371216,-10.022559,-179.354050,-59.260406,-117.882278,-51.119732,-72.913689,-71.733627,-47.036198,-43.166531,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-449.265427,85.414848,-18.753922,15.658614,-5.042217,-6.483629,-13.445410,-15.101054,-13.789881,-12.406620,...,4.472532,4.371707,6.950669,5.499864,3053.851044,1934.289527,0.077862,0.073118,0.030327,0.032584
50%,-414.052704,101.648323,-6.484858,26.062330,3.683270,1.978006,-8.190969,-7.336420,-7.019958,-7.964580,...,5.005613,5.008768,8.319714,7.145524,3711.697319,2283.298100,0.097099,0.088005,0.040157,0.042631
75%,-381.784637,118.712063,5.197469,35.023731,11.247384,10.594362,-2.515469,0.872001,-0.400692,-3.290712,...,5.609456,5.792470,9.884053,9.088214,4554.485841,2714.518369,0.117734,0.101255,0.055869,0.059515
max,-130.888443,243.761856,69.662437,115.599419,54.558987,60.683899,41.940166,63.374218,34.061249,36.054985,...,10.252549,14.992410,21.077323,26.842926,10163.976648,4424.978768,0.475348,0.211934,0.370823,0.382496
